In [8]:
import gzip
import json
import re
from glob import glob

In [9]:

# === Input file ===
template = "/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_*.txt"




In [5]:
# === Helper function ===
def parse_header(lines):
    metadata = {
        "SamplingFrequency": 100.0,  # You can set this based on "2 msec/sample"
        "StartTime": 0.0,  # Can be updated if available
        "Columns": [],
        "Manufacturer": "Biopac Systems Inc.",
    }

    # Infer SamplingFrequency from "2 msec/sample"
    for line in lines:
        if "msec/sample" in line:
            match = re.search(r"(\d+(\.\d+)?)\s*msec/sample", line)
            if match:
                period_ms = float(match.group(1))
                metadata["SamplingFrequency"] = 1000.0 / period_ms

    # Get channel names and units
    channel_names = []
    units = []
    descriptions = {
        "EDA100C-MRI": "Skin conductance level",
        "ECG100C": "Cardiac electrical activity",
        "Digital input": "Trigger or digital event marker",
        "Stimulus": "Stimulus marker",
        "Shock": "Shock delivery marker"
    }

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith("CH"):  # Header ends here
            break
        if line in descriptions:
            ch_name = line.lower().split()[0]
            channel_names.append(ch_name)
            unit = lines[i+1].strip()
            units.append(unit)
            metadata["Columns"].append(ch_name)
            metadata[ch_name] = {
                "Description": descriptions.get(line, ""),
                "Units": unit
            }
            i += 2
        else:
            i += 1

    return metadata, i

In [12]:
for input_file in glob(template):
    print(input_file)
    # === Output files ===
    tsv_gz_file = input_file.replace(".txt", ".tsv.gz")
    json_file = input_file.replace(".txt", ".json")
    
    with open(input_file, "r") as f:
        lines = f.readlines()
    
    # Parse metadata and find where data starts
    metadata, data_start_idx = parse_header(lines)
    
    # Find actual data start (first CH line)
    while not lines[data_start_idx].strip().startswith("CH"):
        data_start_idx += 1
    data_start_idx += 1  # Skip CH header line
    
    # Extract data
    data_lines = lines[data_start_idx:]
    
    # Write TSV.GZ
    with gzip.open(tsv_gz_file, "wt") as gz:
        for line in data_lines:
            gz.write(line)
    
    # Write JSON
    with open(json_file, "w") as jf:
        json.dump(metadata, jf, indent=4)

/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_task-vr_physio.txt
/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_task-happy_physio.txt
/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_rest-rest1_physio.txt
/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_task-egng_physio.txt
/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub-1980/ses-1/perf/sub-1980_ses-1_task-neutral_physio.txt
/Volumes/Current_Projects2-CC0931-Comparative_Medicine/Lab_Projects/PTSD_Subtyping_fMRI_MEG/BIDS_example subjects/BIDS/sub